Перед вами датасет, описывающий поступление детей в колледж.


1. целевая переменная in_college
2. Осмотрите данные.
3. Примените для предобработки пайплайн с прошлого урока.
4. Решите задачу классификации методами:
- логистической регрессии
- линейной классификации
- деревом решений
- методом опорных векторов
5. сравните полученные в каждом решении метрики между собой


Попробуйте использовать GridSearch для поиска оптимальных гиперпараметров, перечисленных выше алгоритмов.


In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error, accuracy_score
import math

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,False,True
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,False,True
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,False,True
3,Vocational,B,Male,Very Interested,Rural,49,6600000,78.2,82.12,True,True
4,Academic,A,Female,Very Interested,Urban,57,5250000,75.1,86.79,False,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   bool   
 10  in_college             1000 non-null   bool   
dtypes: bool(2), float64(2), int64(2), object(5)
memory usage: 72.4+ KB


In [5]:
df['type_school'] = df['type_school'].map({'Academic': 0 ,'Vocational': 1})
df['school_accreditation'] = df['school_accreditation'].map({'A': 0 ,'B': 1})
df['gender'] = df['gender'].map({'Female': 0 ,'Male': 1})
df['residence'] = df['residence'].map({'Urban': 0 ,'Rural': 1})
df['parent_was_in_college'] = df['parent_was_in_college'].map({True: 0 , False: 1})

In [6]:
df.head()

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college
0,0,0,1,Less Interested,0,56,6950000,83.0,84.09,1,True
1,0,0,1,Less Interested,0,57,4410000,76.8,86.91,1,True
2,0,1,0,Very Interested,0,50,6500000,80.6,87.43,1,True
3,1,1,1,Very Interested,1,49,6600000,78.2,82.12,0,True
4,0,0,0,Very Interested,0,57,5250000,75.1,86.79,1,False


In [7]:
h_inter, h_categories = df['interest'].factorize()

In [8]:
h_categories

Index(['Less Interested', 'Very Interested', 'Uncertain', 'Not Interested',
       'Quiet Interested'],
      dtype='object')

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
oh = OneHotEncoder()

In [11]:
coded = oh.fit_transform(h_inter.reshape(-1,1))

In [12]:
coded.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [13]:
proxy = pd.DataFrame(coded.toarray(), columns = ['Less Interested','Very Interested','Uncertain','Not Interested','Quiet Interested'])

In [14]:
df = pd.concat([df, proxy], axis = 1)

In [15]:
df.head()

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college,Less Interested,Very Interested,Uncertain,Not Interested,Quiet Interested
0,0,0,1,Less Interested,0,56,6950000,83.0,84.09,1,True,1.0,0.0,0.0,0.0,0.0
1,0,0,1,Less Interested,0,57,4410000,76.8,86.91,1,True,1.0,0.0,0.0,0.0,0.0
2,0,1,0,Very Interested,0,50,6500000,80.6,87.43,1,True,0.0,1.0,0.0,0.0,0.0
3,1,1,1,Very Interested,1,49,6600000,78.2,82.12,0,True,0.0,1.0,0.0,0.0,0.0
4,0,0,0,Very Interested,0,57,5250000,75.1,86.79,1,False,0.0,1.0,0.0,0.0,0.0


In [16]:
df = df.drop(['interest'], axis = 1)

In [17]:
df.head()

,type_school,school_accreditation,gender,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college,Less Interested,Very Interested,Uncertain,Not Interested,Quiet Interested
0,0,0,1,0,56,6950000,83.0,84.09,1,True,1.0,0.0,0.0,0.0,0.0
1,0,0,1,0,57,4410000,76.8,86.91,1,True,1.0,0.0,0.0,0.0,0.0
2,0,1,0,0,50,6500000,80.6,87.43,1,True,0.0,1.0,0.0,0.0,0.0
3,1,1,1,1,49,6600000,78.2,82.12,0,True,0.0,1.0,0.0,0.0,0.0
4,0,0,0,0,57,5250000,75.1,86.79,1,False,0.0,1.0,0.0,0.0,0.0


In [18]:
X = df.drop(['in_college'], axis = 1)
Y = df['in_college']

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [20]:
X_train.head()

,type_school,school_accreditation,gender,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,Less Interested,Very Interested,Uncertain,Not Interested,Quiet Interested
29,1,1,0,1,50,6450000,78.4,85.86,0,0.0,0.0,1.0,0.0,0.0
535,1,0,1,0,54,5450000,70.3,85.41,1,0.0,1.0,0.0,0.0,0.0
695,1,0,0,0,52,5520000,82.5,84.59,1,0.0,1.0,0.0,0.0,0.0
557,0,0,1,1,41,3020000,50.6,77.18,0,0.0,0.0,0.0,1.0,0.0
836,1,1,1,1,52,6120000,65.8,84.55,0,0.0,0.0,1.0,0.0,0.0


In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
mm = MinMaxScaler()

In [23]:
X_train_mm = mm.fit_transform(X_train)

In [24]:
X_test_mm = mm.transform(X_test)

## Логистическая регрессия

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
logReg = LogisticRegression()

In [27]:
logReg.fit(X_train_mm, Y_train)

LogisticRegression()

In [28]:
logReg.score(X_train_mm,Y_train), logReg.score(X_test_mm, Y_test)

(0.8575, 0.855)

In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
pred_test_logReg = logReg.predict(X_test_mm)

In [31]:
cm_logReg = confusion_matrix (Y_test, pred_test_logReg)
cm_logReg

array([[86, 20],
       [ 9, 85]], dtype=int64)

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [33]:
precision_score(Y_test, pred_test_logReg), recall_score(Y_test, pred_test_logReg), f1_score(Y_test, pred_test_logReg)

(0.8095238095238095, 0.9042553191489362, 0.8542713567839195)

#### Логистическая регрессия с подбором параметров

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.1, 1, 10]
}

In [36]:
grid_search = GridSearchCV(logReg, param_grid=param_grid, cv=5)

In [37]:
grid_search.fit(X_train_mm, Y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_valida

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10],
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [38]:
grid_search.best_params_

{'C': 10, 'penalty': 'l2'}

In [39]:
grid_search.best_score_

0.85125

In [40]:
pred_test_logReg_grid_search = grid_search.predict(X_test_mm)

In [41]:
cm_logReg_grid_search = confusion_matrix (Y_test, pred_test_logReg_grid_search)
cm_logReg_grid_search

array([[87, 19],
       [10, 84]], dtype=int64)

In [42]:
precision_score(Y_test, pred_test_logReg_grid_search), recall_score(Y_test, pred_test_logReg_grid_search), f1_score(Y_test, pred_test_logReg_grid_search)

(0.8155339805825242, 0.8936170212765957, 0.852791878172589)

## Линейная классификация

In [43]:
from sklearn.linear_model import SGDClassifier

In [44]:
SGDcls = SGDClassifier()

In [45]:
SGDcls.fit(X_train_mm, Y_train)

SGDClassifier()

In [46]:
SGDcls.score(X_test_mm, Y_test)

0.86

In [47]:
pred_test_SGD = SGDcls.predict(X_test_mm)

In [48]:
cm_SGD = confusion_matrix (Y_test, pred_test_SGD)
cm_SGD

array([[83, 23],
       [ 5, 89]], dtype=int64)

In [49]:
precision_score(Y_test, pred_test_SGD), recall_score(Y_test, pred_test_SGD),  f1_score(Y_test, pred_test_SGD)

(0.7946428571428571, 0.9468085106382979, 0.8640776699029127)

#### Линейная классификация с подбором параметров

In [50]:
param_grid_SGDcls = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01]
}

In [51]:
grid_search_SGDcls = GridSearchCV(SGDcls, param_grid=param_grid_SGDcls, cv=5)

In [52]:
grid_search_SGDcls.fit(X_train_mm, Y_train)

GridSearchCV(cv=5, estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01],
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [53]:
grid_search_SGDcls.best_params_

{'alpha': 0.001, 'penalty': 'l2'}

In [54]:
grid_search_SGDcls.best_score_

0.8525

In [55]:
pred_test_SGD_grid_search = grid_search_SGDcls.predict(X_test_mm)

In [56]:
cm_SGD_grid_search = confusion_matrix (Y_test, pred_test_SGD_grid_search)
cm_SGD_grid_search

array([[85, 21],
       [ 8, 86]], dtype=int64)

In [57]:
precision_score(Y_test, pred_test_SGD_grid_search), recall_score(Y_test, pred_test_SGD_grid_search), f1_score(Y_test, pred_test_SGD_grid_search)

(0.8037383177570093, 0.9148936170212766, 0.8557213930348259)

## Дерево решений

In [58]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [59]:
param_distributions = {'n_estimators': randint(1, 10), # количество деревьев
                        'max_depth': randint(5, 10)} # глубина деревьев

In [60]:
search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=0),
                             n_iter=5,
                             param_distributions=param_distributions,
                             random_state=0)

search.fit(X_train, Y_train)
search.best_params_

{'max_depth': 9, 'n_estimators': 6}

In [61]:
search.score(X_test, Y_test)

0.7131337486311924

## Метод опорных векторов

In [62]:
from sklearn.svm import SVC, LinearSVC

In [63]:
svm_c = LinearSVC()

In [64]:
svm_c.fit(X_train_mm, Y_train)

LinearSVC()

In [65]:
pred_test_svm = svm_c.predict(X_test_mm)

In [66]:
svm_c.score(X_test_mm, Y_test)

0.865

In [67]:
cm_svm = confusion_matrix (Y_test, pred_test_svm)
cm_svm

array([[87, 19],
       [ 8, 86]], dtype=int64)

In [68]:
precision_score(Y_test, pred_test_svm), recall_score(Y_test, pred_test_svm), f1_score(Y_test, pred_test_svm)

(0.819047619047619, 0.9148936170212766, 0.864321608040201)

#### Метод опорных векторов с подбором параметров

In [69]:
param_grid_LinearSVC = {'C': np.arange(0.1, 5, 0.1)}

In [70]:
grid_search_LinearSVC = GridSearchCV(estimator=svm_c, param_grid=param_grid_LinearSVC, cv=5)

In [71]:
grid_search_LinearSVC.fit(X_train_mm, Y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

GridSearchCV(cv=5, estimator=LinearSVC(),
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9])})

In [72]:
grid_search_LinearSVC.best_params_

{'C': 1.2000000000000002}

In [73]:
grid_search_LinearSVC.best_score_

0.85125

In [74]:
pred_test_LinearSVC_grid_search = grid_search_LinearSVC.predict(X_test_mm)

In [75]:
cm_LinearSVC_grid_search = confusion_matrix (Y_test, pred_test_LinearSVC_grid_search)
cm_LinearSVC_grid_search

array([[87, 19],
       [ 8, 86]], dtype=int64)

In [76]:
precision_score(Y_test, pred_test_LinearSVC_grid_search), recall_score(Y_test, pred_test_LinearSVC_grid_search), f1_score(Y_test, pred_test_LinearSVC_grid_search)

(0.819047619047619, 0.9148936170212766, 0.864321608040201)